In [113]:
from pynq import Overlay
ol = Overlay("overlay.bit")
ol.download()

def send_byte(register, byte):
    # i2c address is 0x76, but axi_iic uses 7 bit addresses, so shift right by 1
    ol.axi_iic_0.send(0x3b, bytes([0x40]) + bytes([register]) + bytes([byte]), 3)

def send_bytes(register, payload):
    ol.axi_iic_0.send(0x3b, bytes([0x40]) + bytes([register]) + bytes(payload), 2 + len(payload))

import time

# R0 (0x00): enable PLL, set input clock frequency to 1024*f_s, disable core clock
send_byte(0x00, 0x0E)

# R1 (0x01): set PLL frequency to (4 + 0.9152)*f_MCLK = 4.9152*10MHz 
# also enable PLL
send_bytes(0x01, [0x02, 0x71, 0x02, 0x3C, 0x21, 0x01])

# wait until PLL lock is achieved
rxdata = bytes(6)
count =  1
while True:
    ol.axi_iic_0.send(0x3b, bytes([0x40, 0x02]), 2)
    ol.axi_iic_0.receive(0x3b, rxdata, 6)
    if rxdata[5] & 0x02 != 0:
        break
    if count > 100:
        print("failed to lock pll")
        break
    count += 1
    time.sleep(0.1)

# R0 (0x00): enable core clock
send_byte(0x00, 0x0F)

# wait for core to start up 
time.sleep(1)

# R15 (0x15): Put CODEC in Master mode
send_byte(0x15, 0x01)

# enable ADC on both channels with normal polarity; enable high-pass antialiasing filter
send_byte(0x19, 0x73)
# enable Mixer 1 (left record channel)
send_byte(0x0A, 0x01)
# enable Mixer 2 (right record channel)
send_byte(0x0C, 0x01)
# set mic boost gain to 20dB
send_byte(0x0B, 0x07)
send_byte(0x0D, 0x07)

# enable ALC control and noise gate
send_byte(0x14, 0x20)

# enable mixer output on L/R channels
# R22 (0x1C): unmute left DAC input to left channel playback mixer
# R24 (0x1E): unmute left DAC input to left channel playback mixer
send_byte(0x1C, 0x21)
send_byte(0x1E, 0x41)

# enable headphone playback on L/R channels
# R29 (0x23): set volume to 0dB (111001xx) and unmute and enable headphone volume control (xxxxxx11)
# R30 (0x24): set volume to 0dB (111001xx) and unmute and enable headphone volume control (xxxxxx11)
send_byte(0x23, 0xA7)
send_byte(0x24, 0xA7)

# Enable playback L/R channels
# R35 (0x29): leave bias control in default normal operation, enable left and right channel playback
send_byte(0x29, 0x03)

# Enable DAC for both channels
# R36 (0x2A): leave in stereo mode, enable both left and right DACs
send_byte(0x2A, 0x03)

# Set DAC/ADC i/o to use i2s data
# R58 (0xF2): set serial input [L0, R0] to DAC L/R
# R59 (0xF3): set ADC L/R to serial output [L0, R0]
send_byte(0xF2, 0x01)
send_byte(0xF3, 0x01)

# R65 (0xF9): enable all digital clocks
send_byte(0xF9, 0x7F)
# R66 (0xFA): enable digital clock generator 0 and 1
send_byte(0xFA, 0x03)

In [114]:
# enable i2s output
ol.axi_gpio_i2s_enable.write(0,1)
# configure fm synth params
ol.axi_gpio_harmonicity.write(0,0x0000)
ol.axi_gpio_mod_index.write(0,0x0000)

In [9]:
# disable i2s output
ol.axi_gpio_i2s_enable.write(0,0)

In [ ]:
import numpy as np
import time

def start_capture():
    ol.axi_dbg_cap.write(0,1)
    ol.axi_dbg_cap.write(0,0)

def dbg_read_next():
    ol.axi_dbg_next.write(0,1)
    ol.axi_dbg_next.write(0,0)
    return ol.axi_dbg_data.read(0)

def dump_frame():
    phase = np.zeros(4096)
    re = np.zeros(4096)
    im = np.zeros(4096)
    max_k_min_n_concat = np.zeros(4096)
    current_phase = np.zeros(4096)
    last_phase = np.zeros(4096)
    for i in range(4096):
        phase[i] = ol.axi_dbg_data_0.read()
        re[i] = ol.axi_dbg_data_1.read()
        im[i] = ol.axi_dbg_data_2.read()
        max_k_min_n_concat[i] = ol.axi_dbg_data_3.read()
        current_phase[i] = ol.axi_dbg_data_4.read()
        last_phase[i] = ol.axi_dbg_data_5.read()
        ol.axi_dbg_next.write(0,1)
        ol.axi_dbg_next.write(0,0)
    return phase, re, im, max_k_min_n_concat, current_phase, last_phase

start_capture()
time.sleep(1)
phase, re, im, max_k_min_n, current_phase, last_phase = dump_frame()

In [ ]:
import matplotlib.pyplot as plt

def hex_to_int(hexnum, bits):
    if hexnum >= 2**(bits - 1):
        return hexnum - 2**bits
    return hexnum

def phase_hex_to_float(hexnum):
    if hexnum >= 0x800000:
        return (hexnum - 2**24)/(2**21)
    return hexnum/(2**21)

re_scaled = [hex_to_int(int(r), 24) for r in re]
im_scaled = [hex_to_int(int(i), 24) for i in im]
mag_scaled = np.zeros(4096)
phase_scaled = np.zeros(4096)
max_k = np.zeros(4096)
min_n = np.zeros(4096)

for i in range(4096):
    mag_scaled[i] = re_scaled[i]**2 + im_scaled[i]**2
    max_k[i] = int(max_k_min_n[i])//(2**6)
    min_n[i] = int(max_k_min_n[i]) & (2**6 - 1)
    if i < 4096-28:
        phase_scaled[i] = phase_hex_to_float(int(phase[i+28]))
        
low_bin = 0
hi_bin = 128
plt.figure()
plt.plot(range(low_bin, hi_bin), re_scaled[low_bin:hi_bin])
plt.plot(range(low_bin, hi_bin), im_scaled[low_bin:hi_bin])
plt.show()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(range(low_bin, hi_bin), mag_scaled[low_bin:hi_bin], color='tab:blue')
ax2.plot(range(low_bin, hi_bin), phase_scaled[low_bin:hi_bin], color='tab:orange')
plt.show()

low_bin = 0
hi_bin = 4096
plt.figure()
plt.plot(range(low_bin, hi_bin), max_k[low_bin:hi_bin])
plt.plot(range(low_bin, hi_bin), min_n[low_bin:hi_bin])
plt.show()

In [78]:
print(bin(int(max_k_min_n[0])))

0b10010001


In [105]:
maxk = 5
start = 0
n = (phase_scaled[maxk+start]-phase_scaled[maxk+start+1024]) + 512/1024*maxk
n_rounded = int(round(n))
print(n_rounded)
fund = (phase_scaled[maxk+start+1024]-phase_scaled[maxk+start]+n_rounded)/(512/48000)
print(maxk/1024*48000)
print(fund)

int(phase[maxk+start] - phase[maxk+start+1024] + 512/1024*(maxk * 2**21))/(2**21)

3
234.375
189.893603325


-4.525531768798828